Реализуем функцию, которая вытаскивает последовательности из fasta файлов

In [1]:
def seq_from_fasta(file_list):
  
    seqs = {}

    for file in file_list:
        row_lines = []
        with open(file) as f:
            for line in f:
                if line[0] == '>':
                    header = line[1:-1]
                else:
                    row_lines.append(line[:-1])
        seq = ''.join(row_lines)
        seqs[header] = seq
    return seqs

In [2]:
dnas = seq_from_fasta(['A0A001.fa', 'A0A0C4Y377.fa'])
seq_1 = list(dnas.values())[0]
seq_2 = list(dnas.values())[1]

#seq_1 = 'ACAGTAG'
#seq_2 = 'ACTCG'

Объявим будущую матрицу скоров

In [3]:
matrix = [[0 for i in range(len(seq_2)+1)] for j in range(len(seq_1)+1)]

и similarity matrix(матрица весов)

In [4]:
sim_matrix = [[0 for i in range(len(seq_2))] for j in range(len(seq_1))]

for i in range(len(seq_1)):
    for j in range(len(seq_2)):
        if seq_1[i] == seq_2[j]: # Если i-ая буква в одной последовательности равна j-ой в другой,
            sim_matrix[i][j] = 0   # тогда ij-ый элемент similarity matrix равен нулю
        else:
            sim_matrix[i][j] = 1   # В противном случае -- равен единице


Введём систему скоров

In [5]:
match = 0     # match не меняет Hamming distance 
mismatch = 1  # mismatch увеличивает на единицу
gap = 1       # gap тоже увеличивает на единицу

Реализуем функцию, которая считает матрицу скоров

In [6]:
def score_matrix(matrix):
    '''
    Считает матрицу скоров всевозможных выравниваний. Так как используем Hamming distance,
    лучшим будет выравнивание с НАИМЕНЬШИМ скором (находится в правом нижнем углу матрицы).
    '''

    for j in range(len(seq_2)+1):
        matrix[0][j] = j*gap
    for i in range(len(seq_1)+1):
        matrix[i][0] = i*gap
    for i in range(1, len(seq_1)+1):
        for j in range(1, len(seq_2)+1):
            matrix[i][j] = min(matrix[i-1][j-1] + sim_matrix[i-1][j-1], matrix[i][j-1] + gap, matrix[i-1][j] + gap)
    return matrix

In [7]:
F = score_matrix(matrix)

Собственно само выравнивание

In [8]:
def align_it(seq_1, seq_2, F, sim_matrix):  
    alignment_1 = ''
    alignment_2 = ''
    i = len(seq_1)
    j = len(seq_2)

    while i > 0 or j > 0:

        if i > 0 and j > 0 and F[i][j] == F[i-1][j-1] + sim_matrix[i-1][j-1]: 

            alignment_1 = seq_1[i-1] + alignment_1   # Если данный скор был получен от элемента по диагонали слева,
            alignment_2 = seq_2[j-1] + alignment_2   # тогда соответствующие буквы остаются в обеих строках выравнивания -- match/mismatch
            i -= 1
            j -= 1

        elif i > 0 and F[i][j] == F[i-1][j] + gap: # Если данный скор получен от элемента сверху,
                                                # тогда в первой строке выравнивания остаётся буква соответствующей строки матрицы, 
            alignment_1 = seq_1[i-1] + alignment_1   # а буква соответствующего столбца "удаляется" из второй строки выравнивания
            alignment_2 = '-' + alignment_2
            i -= 1

        else:
                                              # Если данный скор получен от элемента слева,
            alignment_1 = '-' + alignment_1         # тогда в первой строке вырвнивания "удаляется" буква соотвествующей строки матрицы,
            alignment_2 = seq_2[j-1] + alignment_2  # а во второй строке выравнивания буква соответствующего столбца матрицы остаётся
            j -= 1
            
    final_file = open('output.txt', 'w') # Запишем результат в файл
    final_file.writelines([alignment_1 + '\n', alignment_2])
    final_file.close()
    return 'The output.txt is created'

In [9]:
align_it(seq_1, seq_2, F, sim_matrix)

'The output.txt is created'

In [11]:
blosum = {}
dict = {}
with open("BLOSUM50.txt") as f:
    i = 0
    for line in f:
        if i == 0:
            keys = line.split()
        else:
            key, *values = line.split() # fix here
            for j in range(len(keys)):
                dict[keys[j]] = int(values[j])
            blosum[key] = dict
            dict = {}
        i+=1


In [12]:
str1 = seq_1
str2 = seq_2

In [13]:
#считываем  данные о матрице весов
#blosum = pd.read_excel('BLOSUM50.xlsx', index_col=0)
d=-5
n = len(str1)
m = len(str2)
F = [[0 for i in range(m+1)] for j in range(n+1)]

In [14]:
for i in range(n+1):
    F[i][0] = i*d
for j in range(m+1):
    F[0][j] = j*d

In [15]:
for i in range(1, n+1):

    for j in range(1, m+1):

        F[i][j] = max(F[i-1][j-1] +blosum[str1[i-1]][str2[j-1]] , F[i][j-1]+d, F[i-1][j]+d)

In [16]:
i = n
j = m
A1 = []
B1 = []
while (i!=0)|(j!=0):
    if F[i][j] == F[i-1][j-1]+blosum[str1[i-1]][str2[j-1]]:
        A1.append(str1[i-1])
        B1.append(str2[j-1])
        i-=1
        j-=1
    elif F[i][j] == F[i][j-1]+d:
        A1.append('-')
        B1.append(str2[j-1])
        j-=1
    elif  F[i][j] == F[i-1][j]+d:
        A1.append(str1[i-1])
        B1.append('-')
        i-=1
    

In [19]:
st1 = ''.join(reversed(A1))
st2 = ''.join(reversed(B1))
with open('ou.txt', 'w') as f:
    f.write(st1 + '\n')
    f.write(st2)